In [1]:
# Import necessary libraries
from ultralytics import YOLO
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
from PIL import Image as PILImage
import io
from IPython.display import Image, display
from PIL import Image as PILImage
import numpy as np
import tensorflow as tf
import io
import os
import pandas as pd
import torch
import json
from torch import float32, tensor
import torchmetrics
import cv2


2024-01-14 20:49:02.961036: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 20:49:02.961062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 20:49:02.961655: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from tqdm import tqdm

weights=models.VGG19_Weights.IMAGENET1K_V1
model = models.vgg19(weights=weights)
preprocess = weights.transforms()
model.classifier[6] = nn.Linear(4096, 29)
model.load_state_dict(torch.load('./models/vgg19_model.pth'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
# Load the YOLO model
yolo_model = YOLO('./YOLOv8_training/multiclass_best.pt').to(device)

# Class mapping
class_mapping = {'Apple Scab Leaf': 0, 'Apple leaf': 1, 'Apple rust leaf': 2, 'Bell_pepper leaf': 3, 'Bell_pepper leaf spot': 4, 'Blueberry leaf': 5, 'Cherry leaf': 6, 'Corn Gray leaf spot': 7, 'Corn leaf blight': 8, 'Corn rust leaf': 9, 'Peach leaf': 10, 'Potato leaf': 11, 'Potato leaf early blight': 12, 'Potato leaf late blight': 13, 'Raspberry leaf': 14, 'Soyabean leaf': 15, 'Squash Powdery mildew leaf': 16, 'Strawberry leaf': 17, 'Tomato Early blight leaf': 18, 'Tomato Septoria leaf spot': 19, 'Tomato leaf': 20, 'Tomato leaf bacterial spot': 21, 'Tomato leaf late blight': 22, 'Tomato leaf mosaic virus': 23, 'Tomato leaf yellow virus': 24, 'Tomato mold leaf': 25, 'Tomato two spotted spider mites leaf': 26, 'grape leaf': 27, 'grape leaf black rot': 28}

# Directory containing images to predict
directory = './datasets/test/images'
# directory = './TEST'

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
preds=[]
# Define a list of dictionaries
data = []
# Create empty lists to store results
resized_cropped_images = []
predicted_classes = []

In [5]:
df = pd.read_csv('./YOLOv8_training/new_labels.csv')
# df = pd.read_csv('./set.csv')
model.eval()
# Define the target size for ResNet50
target_size = (224, 224)
# Process YOLO results
for filename in os.listdir(directory):
    filtered_df = df[df['filename'] == filename]
    results = yolo_model.predict(os.path.join(directory,filename))
    for x in results:
        for r in x:
            # Get the image and bounding box
            im_array = r.plot()  # plot a BGR numpy array of predictions
            im = PILImage.fromarray(im_array[..., ::-1])  # Convert BGR to RGB PIL image
            bbox = r.boxes.data[0, :4].cpu().numpy()  # Get the bounding box

            # Crop the detected object from the image
            cropped_im = im.crop((bbox[0], bbox[1], bbox[2], bbox[3]))

            # Resize the cropped image to the target size
            resized_cropped_im = cropped_im.resize(target_size)
            # plt.imshow(resized_cropped_im)

            # Save the resized and cropped image to a BytesIO object
            img_io = io.BytesIO()
            resized_cropped_im.save(img_io, 'PNG')

            # Append the resized and cropped image to the list
            resized_cropped_images.append(resized_cropped_im)
            input_image = preprocess(cropped_im).unsqueeze(0).to(device)
            with torch.no_grad():
                output = model(input_image)
            logits = torch.nn.functional.softmax(output[0], dim=0)
            # model predicts one of the 1000 ImageNet classes
            predicted_class_index = torch.argmax(logits).item()
            predicted_class_label= list(class_mapping.keys())[predicted_class_index]
            predicted_classes.append(predicted_class_label)
            # Convert bounding box to tensor and move to GPU
            bbox_tensor = torch.tensor([[bbox[0], bbox[1], bbox[2], bbox[3]]]).to(device)
            
            # Convert labels to tensor and move to GPU
            label_tensor = torch.tensor([predicted_class_index]).to(device)

            # Convert scores to tensor and move to GPU
            score_tensor = torch.tensor([logits[predicted_class_index].item()]).to(device)*r.boxes.conf
#             score_tensor=r.boxes.conf
            minind=0
            q=0
            err=1e9
            for(index, row) in filtered_df.iterrows():
                # Access fields using the column names
                temerr=(bbox[0]-row['xmin'])**2+(bbox[1]-row['ymin'])**2+(bbox[2]-row['xmax'])**2+(bbox[3]-row['ymax'])**2
                if(temerr<err):
                    err=temerr
                    minind=q
                q+=1
            print(len(filtered_df), " ", minind)
            if(len(filtered_df)>0):
                data.append({'boxes': torch.tensor([[(filtered_df.iloc[minind]['xmin'].copy()), (filtered_df.iloc[minind]['ymin'].copy()), (filtered_df.iloc[minind]['xmax'].copy()), (filtered_df.iloc[minind]['ymax'].copy())]]).to(device), 'labels': torch.tensor([(class_mapping[filtered_df.iloc[minind]['class']])]).to(device)})
                # filtered_df.drop(filtered_df.index[minind], inplace=True)
                preds.append(dict(boxes=bbox_tensor, labels=label_tensor, scores=score_tensor))
    filtered_df=[]
    
    


image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2458.jpg: 480x640 7 Tomato leaf mosaic viruss, 57.2ms
Speed: 0.8ms preprocess, 57.2ms inference, 209.7ms postprocess per image at shape (1, 3, 480, 640)
10   7
10   8
10   2
10   7
10   6
10   9
10   4

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img638.jpg: 576x640 1 Corn leaf blight, 62.3ms
Speed: 1.3ms preprocess, 62.3ms inference, 0.9ms postprocess per image at shape (1, 3, 576, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img454.jpg: 544x640 1 Apple Scab Leaf, 64.5ms
Speed: 1.4ms preprocess, 64.5ms inference, 0.5ms postprocess per image at shape (1, 3, 544, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1086.jpg: 416x640 1 grape leaf black rot, 65.1ms
Speed: 2.4ms preprocess, 65.1ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 

1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1721.jpg: 576x640 1 Tomato leaf, 13 Tomato leaf yellow viruss, 8.0ms
Speed: 1.4ms preprocess, 8.0ms inference, 0.4ms postprocess per image at shape (1, 3, 576, 640)
13   3
13   10
13   2
13   3
13   10
13   10
13   8
13   2
13   9
13   3
13   5
13   8
13   4
13   10

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img247.jpg: 512x640 22 Tomato leaf yellow viruss, 4 Tomato mold leafs, 9.1ms
Speed: 1.1ms preprocess, 9.1ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 640)
5   4
5   2
5   1
5   2
5   2
5   0
5   2
5   2
5   0
5   0
5   2
5   1
5   0
5   3
5   0
5   0
5   0
5   0
5   0
5   2
5   0
5   4
5   4
5   2
5   2
5   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img659.jpg: 512x640 3 Corn leaf blights, 8.5ms
Speed: 0.9ms preprocess, 8.5ms inference, 0.5ms postprocess per image at shape (1, 3

6   3
6   2
6   1
6   5
6   0
6   4
6   2
6   5

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img503.jpg: 640x608 2 Peach leafs, 56.3ms
Speed: 1.4ms preprocess, 56.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 608)
2   0
2   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img973.jpg: 640x640 1 grape leaf, 11.5ms
Speed: 1.3ms preprocess, 11.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1446.jpg: 480x640 12 Blueberry leafs, 9.9ms
Speed: 1.2ms preprocess, 9.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
10   9
10   7
10   0
10   3
10   8
10   1
10   5
10   2
10   6
10   4
10   6
10   2

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img83.jpg: 512x640 5 Tomato leaf yellow viruss, 8.7ms
Speed: 0.9ms preprocess,

Speed: 0.8ms preprocess, 8.0ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img150.jpg: 640x256 1 Corn rust leaf, 59.5ms
Speed: 0.6ms preprocess, 59.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 256)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img763.jpg: 448x640 16 Strawberry leafs, 8.4ms
Speed: 1.4ms preprocess, 8.4ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)
14   3
14   1
14   4
14   13
14   0
14   5
14   8
14   9
14   12
14   6
14   10
14   2
14   7
14   11
14   7
14   12

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img5.jpg: 640x544 1 Cherry leaf, 58.2ms
Speed: 1.3ms preprocess, 58.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 544)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/im

6   0
6   3
6   1
6   2
6   5
6   4

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img962.jpg: 640x480 1 Corn leaf blight, 9.3ms
Speed: 1.0ms preprocess, 9.3ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 480)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img741.jpg: 512x640 3 Strawberry leafs, 8.8ms
Speed: 1.0ms preprocess, 8.8ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)
3   2
3   1
3   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1880.jpg: 448x640 2 Potato leaf early blights, 7.5ms
Speed: 0.9ms preprocess, 7.5ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)
6   0
6   2

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2464.jpg: 448x640 2 Tomato Septoria leaf spots, 7.3ms
Speed: 0.9ms preprocess, 7.3ms inference, 0.4ms postprocess per image at shape (1, 

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2260.jpg: 576x640 1 Apple leaf, 8.7ms
Speed: 1.4ms preprocess, 8.7ms inference, 0.5ms postprocess per image at shape (1, 3, 576, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img57.jpg: 576x640 2 Blueberry leafs, 8.6ms
Speed: 1.1ms preprocess, 8.6ms inference, 0.5ms postprocess per image at shape (1, 3, 576, 640)
2   0
2   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img723.jpg: 480x640 3 Cherry leafs, 9.6ms
Speed: 1.4ms preprocess, 9.6ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
2   1
2   0
2   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img349.jpg: 448x640 2 Apple Scab Leafs, 8.2ms
Speed: 0.9ms preprocess, 8.2ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)
2   0
2   1

image 1/1 /home/suryansh/Documents/GitHub/P

Speed: 0.8ms preprocess, 9.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
7   3
7   5
7   2
7   1
7   0
7   3
7   6
7   0
7   4
7   0
7   4

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img978.jpg: 448x640 2 Tomato leafs, 7.7ms
Speed: 0.9ms preprocess, 7.7ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)
2   0
2   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img950.jpg: 640x640 5 Tomato leaf yellow viruss, 11.2ms
Speed: 1.1ms preprocess, 11.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
5   4
5   1
5   3
5   0
5   2

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img769.jpg: 640x480 3 Corn leaf blights, 9.1ms
Speed: 1.0ms preprocess, 9.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)
2   0
2   1
2   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test

Speed: 1.5ms preprocess, 14.9ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
3   2
3   1
3   0
3   1
3   2
3   2
3   2

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img70.jpg: 384x640 1 Corn rust leaf, 12.3ms
Speed: 0.9ms preprocess, 12.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2291.jpg: 288x640 1 Corn Gray leaf spot, 9.6ms
Speed: 0.8ms preprocess, 9.6ms inference, 1.2ms postprocess per image at shape (1, 3, 288, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1916.jpg: 640x480 2 Corn leaf blights, 13.9ms
Speed: 1.2ms preprocess, 13.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img134.jpg: 448x640 3 Potato leaf early blights, 7.9

Speed: 1.2ms preprocess, 10.5ms inference, 3.7ms postprocess per image at shape (1, 3, 448, 640)
4   0
4   2
4   1
4   3
4   3

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img272.jpg: 416x640 17 Peach leafs, 8.5ms
Speed: 1.0ms preprocess, 8.5ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)
8   5
8   3
8   0
8   4
8   2
8   1
8   7
8   6
8   0
8   1
8   0
8   0
8   6
8   7
8   6
8   0
8   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img584.jpg: 640x544 1 Corn leaf blight, 10.6ms
Speed: 1.3ms preprocess, 10.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 544)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1138.jpg: 448x640 22 Soyabean leafs, 11.5ms
Speed: 0.9ms preprocess, 11.5ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)
11   5
11   3
11   7
11   2
11   1
11   4
11   6
11   9
11   0
11   10
11  

In [6]:
# for img_data, predicted_class in zip(resized_cropped_images, predicted_classes):
#     display(img_data, predicted_class, class_mapping[predicted_class])


In [7]:
metrics = torchmetrics.detection.MeanAveragePrecision().to(device)
for i in range(len(preds)):
    print([preds[i]], [data[i]])
    metrics.update([preds[i]], [data[i]])
print(metrics.compute())

[{'boxes': tensor([[  0.0000, 160.5174, 208.9548, 284.2183]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.3764], device='cuda:0')}] [{'boxes': tensor([[ 25, 160, 204, 279]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}]
[{'boxes': tensor([[428.7725,  90.2564, 565.3612, 248.9724]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.4281], device='cuda:0')}] [{'boxes': tensor([[437,  83, 563, 237]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}]
[{'boxes': tensor([[130.5516,   0.0000, 266.3249, 218.1907]], device='cuda:0'), 'labels': tensor([24], device='cuda:0'), 'scores': tensor([0.1880], device='cuda:0')}] [{'boxes': tensor([[152,   7, 238, 183]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}]
[{'boxes': tensor([[  0.0000,  29.9494, 121.2516, 258.9439]], device='cuda:0'), 'labels': tensor([25], device='cuda:0'), 'scores': tensor([0.3969], device='cuda:0')}] [{'boxes': tensor([[ 25, 1

[{'boxes': tensor([[ 861.1401,  620.5565, 1075.9082,  862.8157]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.9108], device='cuda:0')}] [{'boxes': tensor([[ 864,  636, 1076,  856]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}]
[{'boxes': tensor([[1017.7858,  660.0958, 1229.3063,  878.7527]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.9061], device='cuda:0')}] [{'boxes': tensor([[1023,  662, 1230,  879]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}]
[{'boxes': tensor([[284.1003, 797.6006, 486.9249, 993.7167]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.9050], device='cuda:0')}] [{'boxes': tensor([[ 285,  788,  485, 1001]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}]
[{'boxes': tensor([[ 906.2264,   25.9719, 1149.2333,  326.1736]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.9004], device='cuda:0')}] [{

[{'boxes': tensor([[   4.2443,    5.8913, 1787.3354, 1123.1011]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.5759], device='cuda:0')}] [{'boxes': tensor([[   1,    7, 1811, 1138]], device='cuda:0'), 'labels': tensor([19], device='cuda:0')}]
[{'boxes': tensor([[ 193.7677, 1493.1772, 1554.3322, 2589.8088]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.5541], device='cuda:0')}] [{'boxes': tensor([[ 190, 1274, 1586, 2592]], device='cuda:0'), 'labels': tensor([19], device='cuda:0')}]
[{'boxes': tensor([[  1.1650,   0.0000, 292.0302, 399.6660]], device='cuda:0'), 'labels': tensor([8], device='cuda:0'), 'scores': tensor([0.9112], device='cuda:0')}] [{'boxes': tensor([[  5,  12, 400, 266]], device='cuda:0'), 'labels': tensor([8], device='cuda:0')}]
[{'boxes': tensor([[  0.0000,  43.2039, 347.3562, 215.4951]], device='cuda:0'), 'labels': tensor([19], device='cuda:0'), 'scores': tensor([0.8423], device='cuda:0')}] [{'boxes': t

[{'boxes': tensor([[273.7093, 179.6801, 303.6279, 206.9203]], device='cuda:0'), 'labels': tensor([24], device='cuda:0'), 'scores': tensor([0.3527], device='cuda:0')}] [{'boxes': tensor([[277, 176, 303, 205]], device='cuda:0'), 'labels': tensor([24], device='cuda:0')}]
[{'boxes': tensor([[311.7240, 199.6338, 341.5467, 225.7765]], device='cuda:0'), 'labels': tensor([24], device='cuda:0'), 'scores': tensor([0.1708], device='cuda:0')}] [{'boxes': tensor([[304, 168, 345, 207]], device='cuda:0'), 'labels': tensor([24], device='cuda:0')}]
[{'boxes': tensor([[188.4582, 382.2141, 218.3607, 402.9748]], device='cuda:0'), 'labels': tensor([20], device='cuda:0'), 'scores': tensor([0.1264], device='cuda:0')}] [{'boxes': tensor([[218, 371, 252, 391]], device='cuda:0'), 'labels': tensor([24], device='cuda:0')}]
[{'boxes': tensor([[294.8482, 391.7295, 313.2107, 417.1161]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.0925], device='cuda:0')}] [{'boxes': tensor([[260, 3

[{'boxes': tensor([[128.1531, 278.7531, 370.9870, 358.3388]], device='cuda:0'), 'labels': tensor([0], device='cuda:0'), 'scores': tensor([0.2761], device='cuda:0')}] [{'boxes': tensor([[130, 277, 377, 354]], device='cuda:0'), 'labels': tensor([14], device='cuda:0')}]
[{'boxes': tensor([[ 11.0609,  77.9984, 485.6106, 287.4503]], device='cuda:0'), 'labels': tensor([5], device='cuda:0'), 'scores': tensor([0.5894], device='cuda:0')}] [{'boxes': tensor([[ 17,  81, 488, 282]], device='cuda:0'), 'labels': tensor([5], device='cuda:0')}]
[{'boxes': tensor([[178.1987, 280.9276, 491.0208, 374.9390]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.5148], device='cuda:0')}] [{'boxes': tensor([[ 17,  81, 488, 282]], device='cuda:0'), 'labels': tensor([5], device='cuda:0')}]
[{'boxes': tensor([[  0.6240,  13.9742, 479.9569, 537.3519]], device='cuda:0'), 'labels': tensor([16], device='cuda:0'), 'scores': tensor([0.9607], device='cuda:0')}] [{'boxes': tensor([[ 10,  19, 

[{'boxes': tensor([[405.8697, 314.7092, 480.8093, 458.9401]], device='cuda:0'), 'labels': tensor([18], device='cuda:0'), 'scores': tensor([0.5332], device='cuda:0')}] [{'boxes': tensor([[409, 312, 479, 453]], device='cuda:0'), 'labels': tensor([5], device='cuda:0')}]
[{'boxes': tensor([[330.3932,  94.4704, 430.2958, 137.6842]], device='cuda:0'), 'labels': tensor([18], device='cuda:0'), 'scores': tensor([0.3356], device='cuda:0')}] [{'boxes': tensor([[334,  93, 427, 140]], device='cuda:0'), 'labels': tensor([5], device='cuda:0')}]
[{'boxes': tensor([[431.0196,  42.2141, 500.0000, 120.0941]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.2148], device='cuda:0')}] [{'boxes': tensor([[334,  93, 427, 140]], device='cuda:0'), 'labels': tensor([5], device='cuda:0')}]
[{'boxes': tensor([[143.1840, 242.3254, 259.0680, 300.6587]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.4434], device='cuda:0')}] [{'boxes': tensor([[153, 244,

[{'boxes': tensor([[406.0289, 394.0717, 499.6225, 469.7993]], device='cuda:0'), 'labels': tensor([27], device='cuda:0'), 'scores': tensor([0.1560], device='cuda:0')}] [{'boxes': tensor([[432, 343, 499, 386]], device='cuda:0'), 'labels': tensor([5], device='cuda:0')}]
[{'boxes': tensor([[  4.2150, 461.3423,  69.2760, 499.9014]], device='cuda:0'), 'labels': tensor([24], device='cuda:0'), 'scores': tensor([0.5268], device='cuda:0')}] [{'boxes': tensor([[ 52, 423, 174, 480]], device='cuda:0'), 'labels': tensor([5], device='cuda:0')}]
[{'boxes': tensor([[111.8200, 472.8017, 189.1930, 500.0000]], device='cuda:0'), 'labels': tensor([4], device='cuda:0'), 'scores': tensor([0.2445], device='cuda:0')}] [{'boxes': tensor([[ 52, 423, 174, 480]], device='cuda:0'), 'labels': tensor([5], device='cuda:0')}]
[{'boxes': tensor([[ 14.9528, 341.5471,  54.4576, 435.8614]], device='cuda:0'), 'labels': tensor([24], device='cuda:0'), 'scores': tensor([0.3525], device='cuda:0')}] [{'boxes': tensor([[ 52, 423, 

[{'boxes': tensor([[3.1528e-02, 0.0000e+00, 1.5105e+02, 5.6551e+01]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.0837], device='cuda:0')}] [{'boxes': tensor([[  2,  66, 196, 241]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
[{'boxes': tensor([[  0.0000,   9.4706, 429.8563, 227.8708]], device='cuda:0'), 'labels': tensor([9], device='cuda:0'), 'scores': tensor([0.9081], device='cuda:0')}] [{'boxes': tensor([[  1,  12, 432, 226]], device='cuda:0'), 'labels': tensor([9], device='cuda:0')}]
[{'boxes': tensor([[  0.0000,  37.7491, 658.8502, 265.8933]], device='cuda:0'), 'labels': tensor([8], device='cuda:0'), 'scores': tensor([0.7310], device='cuda:0')}] [{'boxes': tensor([[  1,  35, 660, 269]], device='cuda:0'), 'labels': tensor([7], device='cuda:0')}]
[{'boxes': tensor([[ 272.0979,  290.2258,  815.3485, 1485.4727]], device='cuda:0'), 'labels': tensor([8], device='cuda:0'), 'scores': tensor([0.5674], device='cuda:0')}] [{'boxes': tensor([[

{'map': tensor(0.2845), 'map_50': tensor(0.3443), 'map_75': tensor(0.3261), 'map_small': tensor(0.2529), 'map_medium': tensor(0.2374), 'map_large': tensor(0.3055), 'mar_1': tensor(0.3511), 'mar_10': tensor(0.3511), 'mar_100': tensor(0.3511), 'mar_small': tensor(0.2837), 'mar_medium': tensor(0.2838), 'mar_large': tensor(0.3707), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28], dtype=torch.int32)}
